In [36]:
import dalex as dx
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

In [37]:
data = dx.datasets.load_hr()

# status is the target
X = data.drop(columns='status')
y = data.status

# Transformação da coluna str 'status' que continha (promoted, fired, ou ok) em uma coluna com dados numéricos em que: 0 = ok, 1 = promoted e 2 = fired
status_list = []
for i in y:
    if i == 'fired':
        status_list.append(2)
    if i == 'promoted':
        status_list.append(1)
    if i == 'ok':
        status_list.append(0)

data['status2'] = status_list

y = data.status2
print(data)

categorical_features = ['gender']
numerical_features = ['age', 'hours', 'evaluation', 'salary']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', 'passthrough', numerical_features)
])

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(max_depth=7, random_state=123))
])

clf.fit(X, y)

       gender        age      hours  evaluation  salary    status
1        male  32.582669  41.886256           3       1     fired
2      female  41.211040  36.343392           2       5     fired
3        male  37.705164  36.817179           3       0     fired
4      female  30.060513  38.960317           3       2     fired
5        male  21.102827  62.154639           5       3  promoted
...       ...        ...        ...         ...     ...       ...
9996   female  50.175709  45.582520           5       0  promoted
9997   female  59.087273  40.664725           3       0     fired
9998   female  51.049296  37.810145           4       0     fired
9999     male  36.158740  35.062329           2       3        ok
10000  female  57.962541  54.786236           4       4  promoted

[7847 rows x 6 columns]
       gender        age      hours  evaluation  salary    status  status2
1        male  32.582669  41.886256           3       1     fired        2
2      female  41.211040  36.3433

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['gender']),
                                                 ('num', 'passthrough',
                                                  ['age', 'hours', 'evaluation',
                                                   'salary'])])),
                ('classifier',
                 DecisionTreeClassifier(max_depth=7, random_state=123))])

In [38]:
exp = dx.Explainer(clf, X, y)

Preparation of a new explainer is initiated

  -> data              : 7847 rows 5 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 7847 values
  -> model_class       : sklearn.tree._classes.DecisionTreeClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x105DD738> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0, mean = 0.353, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.987, mean = 0.728, max = 2.0
  -> model_info        : package sklearn

A new explainer has been created!


In [39]:
exp.model_performance().result

,recall,precision,f1,accuracy,auc
DecisionTreeClassifier,0.74161,0.88084,0.805251,0.800881,1.244003
